# Notebook para unificar archivos CSV

En un caso ideal solo haría falta modificar el directorio y el nombre de salida del archivo y se ejecuta.

In [1]:
import pandas as pd
import os

Para indicar que archivos mezclar se debe indicar el directorio donde se encuentran los csv y el nombre del archivo de salida

In [20]:
"""
Modificar las siguientes variables para el script
directorio: es el la ruta al directorio/folder donde se encuentran los csv para ser concatenados.
            Si es un solo archivo solo le pondra headers.
OUTPUT_FILE_NAME: Es el nombre del archivo resultante.
"""
DIRECTORIO = "24 de febrero 2025/datos"
OUTPUT_FILE_NAME = 'merged.csv'

En la siguiente celda se definen las columnas que tendran como header el csv.

In [3]:
columnas_completas = [
    "Timestamp", "Unix Time", "RTC Temp", "GPS UTC Time", "GPS Date", "GPS_Latitude",
    "GPS_Longitude", "GPS Altitude", "GPS Satellites", "GPS HDOP", "GPS Speed (Knots)",
    "GPS Speed (Km/h)", "GPS Track Degrees", "CycleID", "Hash", "SPS30 mc 1.0", "SPS30 mc 2.5",
    "SPS30 mc 4.0", "SPS30 mc 10.0", "SPS30 nc 0.5", "SPS30 nc 1.0", "SPS30 nc 2.5",
    "SPS30 nc 4.0", "SPS30 nc 10.0", "SPS30 Particle Size", "AHT20 Temperature", "AHT20 Humidity",
    "BMP280 Temperature", "BMP280 Pressure","BMP280 Altitude", "co_level"
]


La sigueinte celda concatena los archivos dentro del directorio especificado. Si en un archivo csv hay más columnas que en otro, por ejemplo uno tiene 30 columnas y otro 31, se mezclaran sin ningun problema

In [25]:
def merge_csv(directorio, OUTPUT_FILE_NAME, columnas_completas):
    import csv
    
    dataframes = []

    for archivo in os.listdir(directorio):
        if archivo.endswith(".csv"):
            ruta_completa = os.path.join(directorio, archivo)
            print(f"Procesando: {ruta_completa}")
            
            # Leemos el archivo línea por línea para manejar filas con diferente número de columnas
            filas_procesadas = []
            max_columnas = 0
            
            with open(ruta_completa, 'r', encoding='latin1') as file:
                csv_reader = csv.reader(file)
                for fila in csv_reader:
                    # Actualizamos el máximo número de columnas encontrado
                    max_columnas = max(max_columnas, len(fila))
                    filas_procesadas.append(fila)
            
            print(f"  - Máximo de columnas encontradas: {max_columnas}")
            
            # Normalizamos todas las filas para que tengan el mismo número de columnas
            filas_normalizadas = []
            for fila in filas_procesadas:
                # Si la fila tiene menos columnas, rellenamos con valores vacíos
                while len(fila) < max_columnas:
                    fila.append('')
                filas_normalizadas.append(fila)
            
            # Creamos el dataframe con las filas normalizadas
            columnas_a_usar = columnas_completas[:max_columnas]
            df = pd.DataFrame(filas_normalizadas, columns=columnas_a_usar)
            
            # Reemplazamos strings vacíos con None para consistency
            df = df.replace('', None)
            
            # Agregamos las columnas faltantes (si las hay) con valores nulos
            for col in columnas_completas:
                if col not in df.columns:
                    df[col] = None
            
            # Reordenamos las columnas para que queden en el orden correcto
            df = df[columnas_completas]
            
            dataframes.append(df)

    # Concatenamos todos los dataframes
    if dataframes:
        df_concatenado = pd.concat(dataframes, ignore_index=True)
        df_concatenado.to_csv(OUTPUT_FILE_NAME, index=False)
        print(f"\nArchivo guardado: {OUTPUT_FILE_NAME}")
        print(f"Filas totales: {len(df_concatenado)}")
    else:
        print("No se encontraron archivos CSV para procesar")


In [22]:
merge_csv(DIRECTORIO, OUTPUT_FILE_NAME, columnas_completas)

Procesando: 24 de febrero 2025/datos/merged.csv
  - Columnas encontradas: 30

Archivo guardado: merged.csv
Filas totales: 35455
